In [ ]:
import logging
import os
import sys
import json
import random
from pathlib import Path
from typing import Any, List, Optional
from unittest.mock import MagicMock, patch

import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# Ensure src is in path
project_root = Path.cwd()
if str(project_root / "src") not in sys.path:
    sys.path.append(str(project_root / "src"))

# Matome Imports
from domain_models.config import ProcessingConfig
from domain_models.manifest import Chunk, SummaryNode, DocumentTree, Cluster
from domain_models.types import DIKWLevel
from matome.engines.raptor import RaptorEngine
from matome.engines.interactive_raptor import InteractiveRaptorEngine
from matome.engines.semantic_chunker import JapaneseSemanticChunker
from matome.engines.embedder import EmbeddingService
from matome.engines.cluster import GMMClusterer
from matome.agents.summarizer import SummarizationAgent
from matome.utils.store import DiskChunkStore
from matome.utils.traversal import traverse_source_chunks
from matome.exporters.markdown import export_to_markdown
from matome.exporters.obsidian import ObsidianCanvasExporter

# Configure Logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(name)s - %(levelname)s - %(message)s")
logger = logging.getLogger("UAT")

In [ ]:
# Check for API Key
api_key = os.environ.get("OPENROUTER_API_KEY")
is_mock_mode = not api_key

if is_mock_mode:
    logger.info("⚠️  OPENROUTER_API_KEY not found. Running in MOCK MODE.")
else:
    logger.info("✅  OPENROUTER_API_KEY found. Running in REAL MODE.")

In [ ]:
# Mocking Utilities

def get_mock_embeddings(texts: list[str]):
    # Return numpy array of shape (len(texts), 384)
    return np.array([np.random.rand(384) for _ in texts])

def mock_llm_call(*args, **kwargs):
    return "This is a mock summary generated by the UAT script."

context_managers = []

if is_mock_mode:
    # Mock Embeddings
    # We patch SentenceTransformer within EmbeddingService if possible, or higher level.

    # Mock the SentenceTransformer class itself where it's imported in embedder
    mock_model = MagicMock()
    mock_model.encode.side_effect = lambda texts, **kwargs: get_mock_embeddings(texts)

    p1 = patch("matome.engines.embedder.SentenceTransformer", return_value=mock_model)

    # Mock LLM
    # Patch the SummarizationAgent.summarize method
    p2 = patch("matome.agents.summarizer.SummarizationAgent.summarize", return_value="Mock Summary Content")

    # Also need to mock ChatOpenAI if it's instantiated
    p3 = patch("matome.agents.summarizer.ChatOpenAI", new=MagicMock())

    context_managers.extend([p1, p2, p3])
    for _cm in context_managers:
        _cm.start()

    logger.info("Mocks activated.")

In [ ]:
# Data Generation Logic
test_data_dir = project_root / "test_data"
if not test_data_dir.exists():
    os.makedirs(test_data_dir)
    logger.info(f"Created directory: {test_data_dir}")

# Create sample.txt (Short)
sample_txt_path = test_data_dir / "sample.txt"
if not sample_txt_path.exists():
    dummy_text = "これはテスト用の日本語テキストです。\n" * 100
    with open(sample_txt_path, "w", encoding="utf-8") as _f_gen:
        _f_gen.write(dummy_text)
    logger.info(f"Created dummy file: {sample_txt_path}")

# Create dummy long text (Emin Style)
emin_txt_path = test_data_dir / "エミン流「会社四季報」最強の読み方.txt"
if not emin_txt_path.exists():
    # Generate ~10KB of dummy Japanese text with some structure
    dummy_long_text = ""
    keywords = ["会社四季報", "PSR", "時価総額", "成長株", "バリュー株", "チャート分析", "業績修正", "外国人投資家", "決算発表", "増益"]
    for _idx_gen in range(500):
        sentence = f"第{_idx_gen}章: {random.choice(keywords)}について考える。市場の動向を見る上で{random.choice(keywords)}は重要である。\n"
        dummy_long_text += sentence

    with open(emin_txt_path, "w", encoding="utf-8") as _f_long:
        _f_long.write(dummy_long_text)
    logger.info(f"Created dummy long text file: {emin_txt_path} ({len(dummy_long_text)} chars)")

In [ ]:
# Scenario 1: Quickstart (Data Loading)
logger.info("--- Scenario 1: Quickstart ---")

# Load Text
with open(sample_txt_path, "r", encoding="utf-8") as _f_sc1:
    content_sample = _f_sc1.read()

logger.info(f"Loaded {len(content_sample)} chars from {sample_txt_path.name}")

In [ ]:
# Scenario 1 Continued: Chunking Visualization

# Initialize basic config for chunking
config_chunking = ProcessingConfig(
    embedding_model="sentence-transformers/all-MiniLM-L6-v2",
    chunk_buffer_size=10,
)

# We need an embedder for semantic chunking
embedder_chunking = EmbeddingService(config_chunking)
chunker_simple = JapaneseSemanticChunker(embedder_chunking)

logger.info("Splitting text into chunks...")
chunks_sample = list(chunker_simple.split_text(content_sample, config_chunking))

logger.info(f"Generated {len(chunks_sample)} chunks.")

print("--- First 5 Chunks ---")
for _idx_chunk, c in enumerate(chunks_sample[:5]):
    print(f"[{c.index}] {c.text[:50]}...")

In [ ]:
# Scenario 2: Clustering Deep Dive (Visualization)
logger.info("--- Scenario 2: Clustering Deep Dive ---")

# Generate embeddings for the chunks
texts = [c.text for c in chunks_sample]
embeddings = list(embedder_chunking.embed_strings(texts))
embeddings_np = np.array(embeddings)

logger.info(f"Generated embeddings shape: {embeddings_np.shape}")

# Reduce dimensions to 2D
n_samples = embeddings_np.shape[0]
if n_samples < 2:
    logger.warning(f"Not enough samples for 2D PCA ({n_samples}). Skipping plot.")
    reduced_embeddings = np.zeros((n_samples, 2))
    pca = None
    plot_path = None
else:
    pca = PCA(n_components=2)
    reduced_embeddings = pca.fit_transform(embeddings_np)

    # Plot
    plt.figure(figsize=(10, 6))
    plt.scatter(reduced_embeddings[:, 0], reduced_embeddings[:, 1], alpha=0.7)
    plt.title("Chunk Embeddings Visualization (PCA)")
    plt.xlabel("Component 1")
    plt.ylabel("Component 2")
    plt.grid(True)

    plot_path = project_root / "clustering_plot.png"
    plt.savefig(plot_path)
    logger.info(f"Saved clustering plot to {plot_path}")

In [ ]:
# Scenario 3: Full Raptor Pipeline
logger.info("--- Scenario 3: Full Raptor Pipeline ---")

# Determine input file
target_file = emin_txt_path

with open(target_file, "r", encoding="utf-8") as _f_sc3:
    full_content = _f_sc3.read()

# Initialize Full Engine
# We use a persistent store for the tutorial so we can run the GUI later
db_path = project_root / "tutorials" / "chunks.db"
if db_path.exists():
    try:
        os.remove(db_path)
        logger.info(f"Removed existing DB: {db_path}")
    except Exception as e:
        logger.warning(f"Could not remove existing DB: {e}")

store = DiskChunkStore(db_path=db_path)

config = ProcessingConfig(
    n_clusters=2,
    umap_n_neighbors=2,
    embedding_model="sentence-transformers/all-MiniLM-L6-v2",
    summarization_model="openai/gpt-4o-mini",
    chunk_buffer_size=10,
    max_summary_tokens=200, # Small summaries for speed
)

embedder = EmbeddingService(config)
chunker = JapaneseSemanticChunker(embedder)
clusterer = GMMClusterer()
summarizer = SummarizationAgent(config)

engine = RaptorEngine(
    chunker=chunker,
    embedder=embedder,
    clusterer=clusterer,
    summarizer=summarizer,
    config=config
)

logger.info("Running Raptor Engine...")
try:
    tree = engine.run(full_content, store=store)
    root_id = tree.root_node.id if isinstance(tree.root_node, SummaryNode) else str(tree.root_node.index)
    logger.info(f"Pipeline complete. Root Node ID: {root_id}")
except Exception as e:
    logger.error(f"Pipeline failed: {e}")
    store.close()
    raise

In [ ]:
# Scenario 4: Validation (The "Grok" Moment)
logger.info("--- Scenario 4: Validation (DIKW) ---")

root = tree.root_node
if isinstance(root, SummaryNode):
    # In mock/small data, it might not reach Wisdom level if depth is low
    # But we check whatever level it reached.
    logger.info(f"Root Node Level: {root.level}")
    logger.info(f"Root Node DIKW: {root.metadata.dikw_level}")

    # We ideally want Wisdom, but if text is short it might be Knowledge or Information
    # Assert it's at least Information if we have summary nodes
    assert root.metadata.dikw_level in [DIKWLevel.WISDOM, DIKWLevel.KNOWLEDGE, DIKWLevel.INFORMATION]
    logger.info("✅ Root Node Level Validated")
else:
    logger.warning("Root is a Chunk (text too short for summarization). Skipping DIKW check.")

In [ ]:
# Scenario 5: Interactive Engine & Traversal
logger.info("--- Scenario 5: Interactive Traversal ---")

interactive_engine = InteractiveRaptorEngine(store=store, summarizer=summarizer, config=config)

# Traverse children of root
if isinstance(root, SummaryNode):
    children = list(interactive_engine.get_children(root))
    logger.info(f"Root has {len(children)} children.")

    # Traverse grandchildren
    if children and isinstance(children[0], SummaryNode):
        grandchildren = list(interactive_engine.get_children(children[0]))
        logger.info(f"First child has {len(grandchildren)} children.")
    else:
        logger.info("First child is a Leaf Chunk.")
        grandchildren = []
else:
    children = []
    grandchildren = []

In [ ]:
# Scenario 6: Refinement
logger.info("--- Scenario 6: Interactive Refinement ---")

# Pick a node to refine (preferably a SummaryNode)
target_node = None
for child in children:
    if isinstance(child, SummaryNode):
        target_node = child
        break

if target_node:
    logger.info(f"Refining Node: {target_node.id}")
    logger.info(f"Original Text: {target_node.text[:50]}...")

    refined_node = interactive_engine.refine_node(target_node.id, "Explain simply.")

    logger.info(f"Refined Text: {refined_node.text[:50]}...")
    assert refined_node.metadata.is_user_edited is True
    assert "Explain simply." in refined_node.metadata.refinement_history
    logger.info("✅ Node Refinement Validated")
else:
    logger.warning("No SummaryNode children found to refine.")

In [ ]:
# Scenario 7: Traceability
logger.info("--- Scenario 7: Traceability (Source Chunks) ---")

if target_node:
    source_chunks = list(interactive_engine.get_source_chunks(target_node.id, limit=5))
    logger.info(f"Retrieved {len(source_chunks)} source chunks for node {target_node.id}")

    for sc in source_chunks:
        logger.info(f" - Chunk {sc.index}: {sc.text[:30]}...")

    assert len(source_chunks) > 0
    logger.info("✅ Source Chunk Retrieval Validated")
else:
    logger.warning("Skipping traceability check (no target node).")

In [ ]:
# Scenario 8: Markdown Export
logger.info("--- Scenario 8: Export ---")

md_content = export_to_markdown(tree, store)

output_md_path = project_root / "summary_all.md"
with open(output_md_path, "w", encoding="utf-8") as _f_md:
    _f_md.write(md_content)
logger.info(f"Exported Markdown to {output_md_path}")

In [ ]:
# Scenario 8 Continued: Canvas Export
canvas_exporter = ObsidianCanvasExporter()

output_canvas_path = project_root / "summary_kj.canvas"

# Use export method which writes to file
canvas_exporter.export(tree, output_canvas_path, store)

logger.info(f"Exported Canvas to {output_canvas_path}")

In [ ]:
# Scenario 9: GUI Instructions
logger.info("--- Scenario 9: GUI Launch Instructions ---")
print("\nTo explore the results visually, run the following command in your terminal:")
print("uv run matome serve tutorials/chunks.db")
print("\nThis will launch the Panel-based interactive application.")

In [ ]:
# Cleanup
store.close()
for _cm in context_managers:
    _cm.stop()
logger.info("Mocks deactivated.")
logger.info("🎉 All Systems Go: Matome 2.0 is ready for Knowledge Installation.")